In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from transformers import T5Tokenizer, T5EncoderModel
import os
from os.path import join
import torch
import torch as th
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tqdm.auto import trange
from contextlib import redirect_stdout
import sys
sys.path.append("/n/home12/binxuwang/Github/DiffusionObjectRelation/PixArt-alpha")
from diffusion.model.builder import build_model
from diffusion.utils.misc import set_random_seed, read_config, init_random_seed, DebugUnderflowOverflow
sys.path.append("/n/home12/binxuwang/Github/DiffusionObjectRelation")
from utils.pixart_sampling_utils import pipeline_inference_custom, \
    PixArtAlphaPipeline_custom
from utils.pixart_utils import state_dict_convert
from utils.pixart_utils import construct_diffuser_pipeline_from_config
from utils.cv2_eval_utils import find_classify_objects, evaluate_parametric_relation, eval_func_factory, scene_info_collection
from utils.cv2_eval_utils import evaluate_pipeline_on_prompts, print_evaluation_summary
from utils.relation_shape_dataset_lib import ShapesDataset

/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/n/home12/binxuwang/.conda/envs/torch2/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.i

### Loading the model

In [3]:
model_run_name = "objrel_T5_DiT_B_pilot" # "objrel_rndembdposemb_DiT_B_pilot" 
# model_run_name = "objrel_T5_DiT_mini_pilot_WDecay" # "objrel_rndembdposemb_DiT_B_pilot" 
# ckpt_name = "epoch_4000_step_160000.pth" # "epoch_4000_step_160000.pth" 
ckpt_name = "epoch_1500_step_60000.pth" # "epoch_4000_step_160000.pth" 
text_encoder_type = "T5" 
suffix = ""

In [4]:
savedir = f"/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/results/{model_run_name}"
# savedir = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/results/objrel_rndembdposemb_DiT_B_pilot"
config = read_config(join(savedir, 'config.py'))
# Build pipeline from config
pipeline = construct_diffuser_pipeline_from_config(config, pipeline_class=PixArtAlphaPipeline_custom)

weight_dtype: torch.float16


Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

Expected types for transformer: (<class 'diffusers.models.transformers.pixart_transformer_2d.PixArtTransformer2DModel'>,), got <class 'diffusers.models.transformers.transformer_2d.Transformer2DModel'>.


In [5]:
text_feat_dir_old = '/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/training_datasets/random_emd_dictionary'
T5_path = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/PixArt/output/pretrained_models/t5_ckpts/t5-v1_1-xxl"
tokenizer = T5Tokenizer.from_pretrained(T5_path, )#subfolder="tokenizer")
if text_encoder_type == "T5":
    text_encoder = T5EncoderModel.from_pretrained(T5_path, load_in_8bit=False, torch_dtype=torch.bfloat16, )
    

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
ckptdir = join(savedir, "checkpoints")
ckpt_name = "epoch_1500_step_60000.pth" # "epoch_4000_step_160000.pth" 
ckpt = torch.load(join(ckptdir, ckpt_name))
pipeline.transformer.load_state_dict(state_dict_convert(ckpt['state_dict_ema']))
# pipeline.transformer.load_state_dict(state_dict_convert(ckpt['state_dict']))
pipeline.tokenizer = tokenizer
pipeline.to(device="cuda", dtype=torch.float16);
# pipeline.to(device="cuda", dtype=torch.bfloat16);
pipeline.text_encoder = text_encoder.to(device="cuda", )

/tmp/ipykernel_750701/2026684612.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(join(ckptdir, ckpt_name))


### Generate training set prompts

In [8]:
dataset_tmp = ShapesDataset(num_images=10000)

In [13]:
def generate_prompt_collection(spatial_phrases, 
                               prompt_template="{color1} {shape1} is {rel_text} {color2} {shape2}",
                               color1="blue", shape1="circle", color2="red", shape2="square"):
    prompt_collection = []
    scene_info_collection = []
    for spatial_relationship, rel_text_collection in spatial_phrases.items():
        if spatial_relationship in ["in_front", "behind"]:
            continue
        for rel_text in rel_text_collection:
            prompt = prompt_template.format(color1=color1, shape1=shape1, rel_text=rel_text, color2=color2, shape2=shape2)
            scene_info = {
                "color1": color1,
                "shape1": shape1,
                "color2": color2,
                "shape2": shape2,
                "spatial_relationship": spatial_relationship
            }
            prompt_collection.append(prompt)
            scene_info_collection.append(scene_info)
    return prompt_collection, scene_info_collection

prompt_collection, scene_info_collection = generate_prompt_collection(dataset_tmp.spatial_phrases,
                                prompt_template="{color1} {shape1} is {rel_text} {color2} {shape2}",
                                color1="blue", shape1="circle", color2="red", shape2="square")

In [24]:
from itertools import product
def generate_all_prompt_collection(spatial_phrases, 
                               prompt_template="{color1} {shape1} is {rel_text} {color2} {shape2}",):
    color_list = ['red', 'blue']
    shape_list = ['square', 'triangle', 'circle']
    prompt_collection = []
    scene_info_collection = []
    for color1, color2 in product(color_list, color_list):
        if color1 == color2:      # skip same‐color pairs
            continue
        for shape1, shape2 in product(shape_list, shape_list):
            if shape1 == shape2:
                continue
            for spatial_relationship, rel_text_collection in spatial_phrases.items():
                if spatial_relationship in ["in_front", "behind"]:
                    continue
                for rel_text in rel_text_collection:
                    prompt = prompt_template.format(color1=color1, shape1=shape1, rel_text=rel_text, color2=color2, shape2=shape2)
                    scene_info = {
                        "color1": color1,
                        "shape1": shape1,
                        "color2": color2,
                        "shape2": shape2,
                        "spatial_relationship": spatial_relationship
                    }
                    prompt_collection.append(prompt)
                    scene_info_collection.append(scene_info)
    return prompt_collection, scene_info_collection

In [ ]:
prompt_collection, scene_info_collection = generate_all_prompt_collection(dataset_tmp.spatial_phrases,
                                prompt_template="{color1} {shape1} is {rel_text} {color2} {shape2}",)

In [26]:
len(prompt_collection)

264

### Epoch 1500

In [17]:
ckpt_name = "epoch_1500_step_60000.pth" # "epoch_4000_step_160000.pth" 
ckpt = torch.load(join(ckptdir, ckpt_name))
pipeline.transformer.load_state_dict(state_dict_convert(ckpt['state_dict_ema']))

/tmp/ipykernel_750701/21473023.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(join(ckptdir, ckpt_name))


<All keys matched successfully>

In [ ]:
prompt_collection, scene_info_collection = generate_prompt_collection(dataset_tmp.spatial_phrases,
                                prompt_template="{color1} {shape1} is {rel_text} {color2} {shape2}",
                                color1="blue", shape1="circle", color2="red", shape2="square")

pipeline.set_progress_bar_config(disable=True)
eval_df_syn, object_df_syn = evaluate_pipeline_on_prompts(pipeline, prompt_collection, scene_info_collection,
                                                          num_images=49, num_inference_steps=14, guidance_scale=4.5,
                                                          max_sequence_length=20, generator_seed=42, prompt_dtype=torch.float16,
                                                          color_margin=25, spatial_threshold=5, device="cuda")

print_evaluation_summary(eval_df_syn, group_by_prompt=False)

Evaluating prompts:   0%|          | 0/22 [00:00<?, ?it/s]

torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([9

In [15]:
prompt_collection, scene_info_collection = generate_prompt_collection(dataset_tmp.spatial_phrases,
                                prompt_template="{color1} {shape1} {rel_text} {color2} {shape2}",
                                color1="blue", shape1="circle", color2="red", shape2="square")

pipeline.set_progress_bar_config(disable=True)
eval_df_syn, object_df_syn = evaluate_pipeline_on_prompts(pipeline, prompt_collection, scene_info_collection,
                                                          num_images=49, num_inference_steps=14, guidance_scale=4.5,
                                                          max_sequence_length=20, generator_seed=42, prompt_dtype=torch.float16,
                                                          color_margin=25, spatial_threshold=5, device="cuda")

print_evaluation_summary(eval_df_syn, group_by_prompt=False)

Evaluating prompts:   0%|          | 0/22 [00:00<?, ?it/s]

torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([9

In [16]:
prompt_collection, scene_info_collection = generate_prompt_collection(dataset_tmp.spatial_phrases,
                                prompt_template="{color1} {shape1} {rel_text} the {color2} {shape2}",
                                color1="blue", shape1="circle", color2="red", shape2="square")

pipeline.set_progress_bar_config(disable=True)
eval_df_syn, object_df_syn = evaluate_pipeline_on_prompts(pipeline, prompt_collection, scene_info_collection,
                                                          num_images=49, num_inference_steps=14, guidance_scale=4.5,
                                                          max_sequence_length=20, generator_seed=42, prompt_dtype=torch.float16,
                                                          color_margin=25, spatial_threshold=5, device="cuda")

print_evaluation_summary(eval_df_syn, group_by_prompt=False)

Evaluating prompts:   0%|          | 0/22 [00:00<?, ?it/s]

torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([9

### Epoch 4000

In [18]:
ckpt_name = "epoch_4000_step_160000.pth"  # "epoch_4000_step_160000.pth" 
ckpt = torch.load(join(ckptdir, ckpt_name))
pipeline.transformer.load_state_dict(state_dict_convert(ckpt['state_dict_ema']))


/tmp/ipykernel_750701/187202419.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(join(ckptdir, ckpt_name))


<All keys matched successfully>

In [20]:
prompt_collection, scene_info_collection = generate_prompt_collection(dataset_tmp.spatial_phrases,
                                prompt_template="{color1} {shape1} is {rel_text} {color2} {shape2}",
                                color1="blue", shape1="circle", color2="red", shape2="square")

pipeline.set_progress_bar_config(disable=True)
eval_df_syn, object_df_syn = evaluate_pipeline_on_prompts(pipeline, prompt_collection, scene_info_collection,
                                                          num_images=49, num_inference_steps=14, guidance_scale=4.5,
                                                          max_sequence_length=20, generator_seed=42, prompt_dtype=torch.float16,
                                                          color_margin=25, spatial_threshold=5, device="cuda")

print_evaluation_summary(eval_df_syn, group_by_prompt=False)

Evaluating prompts:   0%|          | 0/22 [00:00<?, ?it/s]

torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([9

In [19]:
prompt_collection, scene_info_collection = generate_prompt_collection(dataset_tmp.spatial_phrases,
                                prompt_template="{color1} {shape1} {rel_text} {color2} {shape2}",
                                color1="blue", shape1="circle", color2="red", shape2="square")

pipeline.set_progress_bar_config(disable=True)
eval_df_syn, object_df_syn = evaluate_pipeline_on_prompts(pipeline, prompt_collection, scene_info_collection,
                                                          num_images=49, num_inference_steps=14, guidance_scale=4.5,
                                                          max_sequence_length=20, generator_seed=42, prompt_dtype=torch.float16,
                                                          color_margin=25, spatial_threshold=5, device="cuda")

print_evaluation_summary(eval_df_syn, group_by_prompt=False)

Evaluating prompts:   0%|          | 0/22 [00:00<?, ?it/s]

torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([9

In [21]:
prompt_collection, scene_info_collection = generate_prompt_collection(dataset_tmp.spatial_phrases,
                                prompt_template="{color1} {shape1} {rel_text} the {color2} {shape2}",
                                color1="blue", shape1="circle", color2="red", shape2="square")

pipeline.set_progress_bar_config(disable=True)
eval_df_syn, object_df_syn = evaluate_pipeline_on_prompts(pipeline, prompt_collection, scene_info_collection,
                                                          num_images=49, num_inference_steps=14, guidance_scale=4.5,
                                                          max_sequence_length=20, generator_seed=42, prompt_dtype=torch.float16,
                                                          color_margin=25, spatial_threshold=5, device="cuda")

print_evaluation_summary(eval_df_syn, group_by_prompt=False)

Evaluating prompts:   0%|          | 0/22 [00:00<?, ?it/s]

torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([9

In [22]:
prompt_collection, scene_info_collection = generate_prompt_collection(dataset_tmp.spatial_phrases,
                                prompt_template="the {color1} {shape1} {rel_text} the {color2} {shape2}",
                                color1="blue", shape1="circle", color2="red", shape2="square")

pipeline.set_progress_bar_config(disable=True)
eval_df_syn, object_df_syn = evaluate_pipeline_on_prompts(pipeline, prompt_collection, scene_info_collection,
                                                          num_images=49, num_inference_steps=14, guidance_scale=4.5,
                                                          max_sequence_length=20, generator_seed=42, prompt_dtype=torch.float16,
                                                          color_margin=25, spatial_threshold=5, device="cuda")

print_evaluation_summary(eval_df_syn, group_by_prompt=False)

Evaluating prompts:   0%|          | 0/22 [00:00<?, ?it/s]

torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([98, 20])
torch.Size([98, 20, 4096])
torch.Size([9

In [27]:
prompt_collection, scene_info_collection = generate_all_prompt_collection(dataset_tmp.spatial_phrases,
                                prompt_template="{color1} {shape1} is {rel_text} {color2} {shape2}",)
pipeline.set_progress_bar_config(disable=True)
eval_df_syn, object_df_syn = evaluate_pipeline_on_prompts(pipeline, prompt_collection, scene_info_collection,
                                                          num_images=49, num_inference_steps=14, guidance_scale=4.5,
                                                          max_sequence_length=20, generator_seed=42, prompt_dtype=torch.float16,
                                                          color_margin=25, spatial_threshold=5, device="cuda")

print_evaluation_summary(eval_df_syn, group_by_prompt=False)

Evaluating prompts:   0%|          | 0/264 [00:00<?, ?it/s]

=== EVALUATION SUMMARY ===
Total samples evaluated: 12936
Number of prompts: 264

--- Overall Performance ---
overall                  : 0.749
overall_loose            : 0.809
shape                    : 0.856
color                    : 0.893
exist_binding            : 0.821
unique_binding           : 0.820
spatial_relationship     : 0.749
spatial_relationship_loose: 0.809
Dx                       : -0.856
Dy                       : -0.422
x1                       : 64.531
y1                       : 64.164
x2                       : 65.387
y2                       : 64.587


In [28]:
prompt_collection, scene_info_collection = generate_all_prompt_collection(dataset_tmp.spatial_phrases,
                                prompt_template="{color1} {shape1} is {rel_text} the {color2} {shape2}",)
pipeline.set_progress_bar_config(disable=True)
eval_df_syn, object_df_syn = evaluate_pipeline_on_prompts(pipeline, prompt_collection, scene_info_collection,
                                                          num_images=49, num_inference_steps=14, guidance_scale=4.5,
                                                          max_sequence_length=20, generator_seed=42, prompt_dtype=torch.float16,
                                                          color_margin=25, spatial_threshold=5, device="cuda")

print_evaluation_summary(eval_df_syn, group_by_prompt=False)

Evaluating prompts:   0%|          | 0/264 [00:00<?, ?it/s]

=== EVALUATION SUMMARY ===
Total samples evaluated: 12936
Number of prompts: 264

--- Overall Performance ---
overall                  : 0.302
overall_loose            : 0.494
shape                    : 0.914
color                    : 0.930
exist_binding            : 0.893
unique_binding           : 0.891
spatial_relationship     : 0.302
spatial_relationship_loose: 0.494
Dx                       : -33.754
Dy                       : -25.145
x1                       : 48.676
y1                       : 47.181
x2                       : 82.431
y2                       : 72.326


In [29]:
prompt_collection, scene_info_collection = generate_all_prompt_collection(dataset_tmp.spatial_phrases,
                                prompt_template="{color1} {shape1} {rel_text} the {color2} {shape2}",)
pipeline.set_progress_bar_config(disable=True)
eval_df_syn, object_df_syn = evaluate_pipeline_on_prompts(pipeline, prompt_collection, scene_info_collection,
                                                          num_images=49, num_inference_steps=14, guidance_scale=4.5,
                                                          max_sequence_length=20, generator_seed=42, prompt_dtype=torch.float16,
                                                          color_margin=25, spatial_threshold=5, device="cuda")

print_evaluation_summary(eval_df_syn, group_by_prompt=False)

Evaluating prompts:   0%|          | 0/264 [00:00<?, ?it/s]

=== EVALUATION SUMMARY ===
Total samples evaluated: 12936
Number of prompts: 264

--- Overall Performance ---
overall                  : 0.218
overall_loose            : 0.403
shape                    : 0.942
color                    : 0.959
exist_binding            : 0.929
unique_binding           : 0.927
spatial_relationship     : 0.218
spatial_relationship_loose: 0.403
Dx                       : -35.012
Dy                       : -37.074
x1                       : 47.970
y1                       : 42.555
x2                       : 82.982
y2                       : 79.629
